# Tracking the Properties of Real Packets

**TARDIS** has the functionality to track the properties of the *RPackets* that are generated when running the Simulation. The `rpacket_tracker` can track all the interactions a packet undergoes & thus keeps a track of the various properties, a packet may have.<br>The `rpacket_tracker` tracks the properties of all the rpackets in the *All the Iterations of the Simulation*. It generates a `pandas.DataFrame` that contains the the properties of all the interactions that a particular `RPacket` undergoes as it is propagated throughout the run of the simulation. This happens for all the packets of each respective `Iteration` and are stored along with the `iteration number`. A sample of the *RPacket Tracking DataFrame* can be seen in the end of this tutorial.

<html>
    <body>
        The properties that are tracked are as follows :
        <ol>
            <li><b>index</b> - Index of the Packet</li>
            <li><b>seed</b> - Seed of the Packet</li>
            <li><b>status</b> - Current Status for the Packet Interaction</li>
            <li><b>r</b> - Radius of the Current Shell</li> 
            <li><b>nu</b> - Packet's Frequency</li> 
            <li><b>mu</b> - Propagation Direction of the Packet (cosine of the angle the packet’s path makes with the radial direction)</li>
            <li><b>energy</b> - Energy of the Packet</li>
            <li><b>shell_id</b> - Current Shell Id where the Packet is present</li>
        </ol>
    </body>
</html>

<div class="alert alert-warning">

Warning

Turning on the `tracking` option in the config has consequences on the time of the simulation run. Please keep this into consideration as the time for processing the output is high and can lead to long simulation times for simpler runs as well.<br>Properties can be accessed through the `DataFrame` indexing easily.
</div>

## How to Setup the Tracking for the RPackets?

**TARDIS**' `rpacket_tracker` is configured via the `YAML` file. This functionality of tracking the packets is turned **off**, by default. This is due to the fact that using this property, leads to longer execution time for the Simulation. An example configuration can be seen below for setting up the *tracking*:

```yaml
... 
montecarlo:
...
tracking:
    track_rpacket: true
```

The `montecarlo` section of the **YAML** file has a `tracking` sub section which holds the configuration properties for the `track_rpacket` & the `initial_array_length` (discussed later in the tutorial).

Let us see, the new `rpacket_tracker` in action.

In [ ]:
from tardis.io.config_reader import Configuration

In [ ]:
# Reading the Configuration stored in `tardis_config_packet_tracking.yml` into config

config = Configuration.from_yaml("tardis_tracking_example.yml")

In [ ]:
# Checking the `tracking` section via the Schema

config["montecarlo"]["tracking"]

In [ ]:
# Setting `r_packet_tracking` to True to turn on the Tracking 

config["montecarlo"]["tracking"]["track_rpacket"] = True

In [ ]:
config["montecarlo"]["tracking"]

In [ ]:
from tardis import run_tardis

In [ ]:
# Running the simulation from the config

sim = run_tardis(config, log_level="Debug", show_convergence_plots=False, show_progress_bars=False)

Now, the `tracked` properties can be accessed via the `rpacket_tracker` attribute of the `sim.runner` object. 

In [ ]:
type(sim.runner.rpacket_tracker)

It can be seen from the above code, that the `sim.runner.rpacket_tracker` is an instance of the `pandas.DataFrame` object. The `RPacketCollection` class has the following structure for the properties : {More information in the **TARDIS API** for `RPacketCollection` class}

```python
# Basic structure for the RPacketCollection Class
class RPacketCollection:
   # Properties
    index
    seed
    status
    r
    nu
    mu
    energy
    shell_id
```

In [ ]:
len(sim.runner.rpacket_tracker)

The generated DataFrame can be accessed with `sim.runner.rpacket_tracker`. The DataFrame for this particular *simulation configuration* can be seen as follows :

In [ ]:
sim.runner.rpacket_tracker

To access these different properties, we may consider the following examples for the `rpacket_tracker`:
<br>In this Example, we are trying to access the properties of the packet at index `10`.<br>In a similar way, we can check for any property for any packet in the range of packets for the last iteration.

- Accessing the `index` property for the packets :

In [ ]:
sim.runner.rpacket_tracker["Packet Index"]

- Accessing the `seed` property for the packets :

In [ ]:
sim.runner.rpacket_tracker["Packet Seed"].unique()

- Accessing the `status` property for the packets :

In [ ]:
sim.runner.rpacket_tracker["Packet Status"]

In [ ]:
sim.runner.rpacket_tracker.loc[sim.runner.rpacket_tracker["Iteration"] == 1]

In [ ]:
sim.runner.rpacket_tracker.loc[sim.runner.rpacket_tracker["Iteration"] == 9]

Thus, all other properties {`r`, `nu`, `mu`, `energy`, `shell_id`} can be accessed accordingly.

<div class="alert alert-warning">

Warning

If we  try to access `sim.runner.rpacket_tracker` property when we have the `track_rpacket` property in the `tracking` subsection of `montecarlo` config, turned off as follows `config["montecarlo"]["tracking"]["track_rpacket"] = False`, it will return `None`.  Error will be raised if we try to access the properties i.e. `seed`, `index`, etc.
</div>

<div class="alert alert-info">

Note
    
When we initialise the `RPacketCollection()` class, the properties arrays {`index`, `seed`, `status`, etc} are allocated certain length based on the `initial_array_length` parameter that can be set via the `initial_array_length` property under `montecarlo -> tracking` section of the configuration. The default size of the array is `10`. This variable is important as the number of interactions a packet may have is variable, thus we need to allocate space dynamically. This variable is used to compute the size and expand the array such that the properties are able to hold these values for the packet interaction. Higher number, allocates more space initially leading to lesser times the arrays expands and vice versa. It can be set in the following manner `config["montecarlo"]["tracking"]["initial_array_length"] = {value}`.
</div>